In [1]:
import os
import re
import json
from tqdm import tqdm

import pandas as pd

from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import get_openai_callback
from langchain_core.messages.system import SystemMessage
from langchain_core.messages.human import HumanMessage
from datasets import load_dataset

In [2]:
CAMBIO_DOLAR = 5.41
MODEL_NAME = "gpt-4o-mini"
INPUT_PRICE_PER_MILLION = 0.15
OUTPUT_PRICE_PER_MILLION = 0.6
FOLDER_PATH = "data/question-generator/base-train-dataset-raw"

llm = ChatOpenAI(model=MODEL_NAME)

prompt_text = """
Dado o seguinte contexto:
```
{context}
```
Escreva um conjunto de perguntas e respostas sobre o contexto.
As perguntas devem ser tais que, concatenando as respostas, seja possível recuperar o conteúdo completo do contexto.
As perguntas deve ser autocontidas; ou seja, um leitor deve entendê-la sem neessariamente conhecer o contexto



# Formato de saída:
O resultado deve ser parseavel em json:

```
[
 {{"pergunta": "<<pergunta 1>>", "pergunta": "<<resposta 1>>"}},
 {{"pergunta": "<<pergunta 2>>", "pergunta": "<<resposta 2>>"}},
 ...
]
```
"""



In [3]:
def get_list(texto):
    resultado = re.search(r'\[([\s\S]*)\]', texto)
    if resultado:
        conteudo = resultado.group(1)
        return json.loads(f"[{conteudo}]")
    else:
        return "Nenhum conteúdo encontrado entre os colchetes."


In [4]:
def get_questions(context):
    chat_prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=prompt_text.format(context=context))
    ])
    
    chain = chat_prompt | llm
    with get_openai_callback() as cb:
        response = chain.invoke(dict(context=context))
        preco_em_real = CAMBIO_DOLAR * (INPUT_PRICE_PER_MILLION * cb.prompt_tokens+ OUTPUT_PRICE_PER_MILLION*cb.completion_tokens) / 1_000_000
        # print(f"preco = {preco_em_real}")

    
    try:
        _list = get_list(response.content)
        error = False
    except:
        _list = None
        error = True
        
    return response.content, _list, preco_em_real, error


In [5]:
def run_llm_data_generator(row):
    response, formatted_response, preco_real, error = get_questions(row.context)

    df = pd.DataFrame(
        [
            {
                "id": row.id,
                "context": row.context,
                "questions": response,
                "formatted_questions": formatted_response,
                "price": preco_real,
            }
        ]
    )

    df.to_parquet(f"{FOLDER_PATH}/{row.id}.parquet")
    return response, preco_real, error

In [6]:
import os
import pandas as pd
import pyarrow.parquet as pq

def read_parquets(folder_path):

    dataframes = []
    
    for arquivo in os.listdir(folder_path):
        if arquivo.endswith('.parquet'):
            try:
                caminho_arquivo = os.path.join(folder_path, arquivo)
                df = pq.read_table(caminho_arquivo).to_pandas()
                dataframes.append(df)
            except:
                print(f"[ERROR] {caminho_arquivo}")

    return pd.concat(dataframes, ignore_index=True)


# Loop

In [7]:
dataset = load_dataset("nunorc/squad_v1_pt")
dataset = dataset["train"].shuffle()


In [8]:
df = dataset.to_pandas()
df["context_len"] = df["context"].str.len()
df = df[df["context_len"] < 1000]

df

,id,title,context,question,answers,context_len
0,56de9b07cffd8e1900b4ba2d,Materialism,"Na antiga filosofia indiana, o materialismo se...",Qual escola avançou o atomismo?,"{'text': ['Escola Nyaya-Vaisesika'], 'answer_s...",522
1,56e7b45837bdd419002c43aa,Arena_Football_League,"A partir de 2014, a ESPN retornou à AFL como p...",Quem foi o parceiro de transmissão da Arena Fo...,"{'text': ['ESPN'], 'answer_start': [20]}",432
3,572bf16ef182dd1900d7c799,Tennessee,Os verões no estado são geralmente quentes e ú...,Qual é a temperatura mais alta já registrada n...,"{'text': ['113 ° F (45 ° C)'], 'answer_start':...",550
4,5726b245dd62a815002e8d27,Crimean_War,A guerra começou nos Bálcãs quando as tropas r...,"Quando os turcos tentaram fornecer reforços, o...","{'text': ['Sinop'], 'answer_start': [407]}",818
5,572777c4708984140094de61,Heian_period,Os detentores de Shōen tinham acesso a mão de ...,Qual classe se tornou uma nova elite militar?,"{'text': ['classe alta provincial'], 'answer_s...",568
...,...,...,...,...,...,...
87591,57284e224b864d190016490d,History_of_India,Os séculos VII e VI aC testemunharam a composi...,Qual parte do sistema de crenças do hinduísmo ...,"{'text': ['ritual'], 'answer_start': [258]}",569
87594,57302410947a6a140053d176,"Tucson,_Arizona",O sudeste de Tucson continua a experimentar um...,Rita Ranch também tem um desenvolvimento plane...,"{'text': ['Civano'], 'answer_start': [709]}",831
87595,572fe742947a6a140053cdcf,Premier_League,A Premier League é transmitida nos Estados Uni...,Qual foi o valor dessa extensão?,{'text': ['em um negócio avaliado em US $ 1 bi...,572
87596,57267c915951b619008f7462,The_Sun_(United_Kingdom),Aproveitando a oportunidade para aumentar sua ...,Qual foi o preço pago pelo jornal?,"{'text': ['£ 800.000'], 'answer_start': [391]}",554


In [9]:
%%time

SAMPLE_SIZE = 10
MAX_TOTAL_PRICE = 15.
MAX_ERROR_COUNT = int(0.1*SAMPLE_SIZE)


sample = df.sample(SAMPLE_SIZE)

errors_count = 0

total_price = read_parquets(FOLDER_PATH)['price'].sum()


for index, row in tqdm(sample.iterrows()):
    if (errors_count > MAX_ERROR_COUNT) or (total_price > MAX_TOTAL_PRICE):
        print(f"Forcing break")
        print(f"errors_count = {errors_count} | MAX_ERROR_COUNT = {MAX_ERROR_COUNT} | errors_count >= MAX_ERROR_COUNT =", errors_count >= MAX_ERROR_COUNT)
        print(f"total_price = {total_price} | MAX_TOTAL_PRICE = {MAX_TOTAL_PRICE} | total_price >= MAX_TOTAL_PRICE =", total_price >= MAX_TOTAL_PRICE)
        break
    
    try:
        print(row["id"])
        response, preco_real, error = run_llm_data_generator(row)
        total_price += preco_real
        if error:
            errors_count += 1
    except Exception as err:
        err_message = f"[ERRO] {err}"
        print(err_message)
        if "Incorrect API key provided" in err_message:
            print(f"Forcing break")
            break
        errors_count += 1

    if (index+1) % 10 == 0:
        try:
            temp = read_parquets(FOLDER_PATH)
            print(f"Preço atá agora: {temp['price'].sum()}")
        except:
            print("Não foi possível ler o parquet. Corrija isso antes de prosseguir")
            break


0it [00:00, ?it/s]

5727d72bff5b5019007d96a4


1it [00:02,  2.70s/it]

5730374704bcaa1900d7738f


2it [00:06,  3.20s/it]

...

Preço atá agora: 9.190218024
570983a2200fba14003680f7


9it [00:48,  6.05s/it]

572833243acd2414000df6cd


10it [00:50,  5.08s/it]

CPU times: user 23 s, sys: 2.98 s, total: 26 s
Wall time: 57.6 s


In [45]:
import json
response = read_parquets(FOLDER_PATH)
response = response[~response["formatted_questions"].isna()]
response = response.rename(columns={"formatted_questions": "questions_and_answers"})
response["questions_and_answers"] = response["questions_and_answers"].apply(lambda x: json.dumps(list(x), ensure_ascii=False))
print(f"Preço total: {response['price'].sum()}")
response = response[["id", "context", "questions_and_answers"]]
response.to_parquet(FOLDER_PATH.replace("base-train-dataset-raw", "base-train-dataset")+".parquet")

response

Preço total: 9.188689157999999


,id,context,questions_and_answers
0,57328ad357eb1f1400fd2d9a,"Em contraste com o Proterozóico, as rochas arq...","[{""pergunta"": ""Quais são as características da..."
1,57279b10ff5b5019007d90d4,Outros estudiosos [quem?] Sugerem que esses ar...,"[{""pergunta"": ""Quais são as críticas feitas po..."
2,56e78da500c9c71400d772a7,"Zhu Yousong, no entanto, se saiu muito pior do...","[{""pergunta"": ""Quem foi Zhu Yousong em relação..."
3,5727266ff1498d1400e8f416,Como a memória da &quot;Carga da Brigada de Lu...,"[{""pergunta"": ""Qual evento histórico é mencion..."
4,56d00d6d234ae51400d9c2e8,O Partido Democrata detém a maioria dos cargos...,"[{""pergunta"": ""Qual partido político detém a m..."
...,...,...,...
7004,56d114ae17492d1400aab8bb,O transporte público é essencial na cidade de ...,"[{""pergunta"": ""Qual a porcentagem de nova-iorq..."
7005,572fa578947a6a140053cb04,"Nos últimos anos, houve uma alta demanda por b...","[{""pergunta"": ""Qual é a alta demanda observada..."
7006,5728a3732ca10214002da504,A missão declarada da BYU &quot;é ajudar as pe...,"[{""pergunta"": ""Qual é a missão declarada da BY..."
7007,56df740d56340a1900b29ba2,O presidente Bush negou financiamento ao UNFPA...,"[{""pergunta"": ""Qual presidente negou financiam..."
